In [ ]:
from gurobipy import *
import json

with open("../Excercises/island.json") as json_file:
    data = json.load(json_file)

distances={}
for key,val in data["distances"].items():
    distances[(int(key.split()[0]),int(key.split()[1]))]=val
    
connections=tuplelist(distances)

m=Model()

x=m.addVars(connections)

for key,val in distances.items():
    if(val>5*3600):
        m.addConstr(x[key]==0)
        
for pos in range(data["numberOfPlaces"]):
    if pos == data["start"]:
        m.addConstr(x.sum(pos,'*')==1+x.sum('*',pos))
    elif pos == data["destination"]:
        m.addConstr(x.sum('*',pos)==1+x.sum(pos,'*'))
    else:
        m.addConstr(x.sum('*',pos)==x.sum(pos,'*'))
        
m.setObjective(quicksum([x[key]*val for key,val in distances.items()]),GRB.MINIMIZE)
#m.setObjective(x.sum(),GRB.MINIMIZE)

m.optimize()

con_list = [con for con in connections if x[con].x>0]
print(con_list)

In [ ]:
import networkx as nx
G = nx.DiGraph()
G.add_nodes_from([i for i in range(data["numberOfPlaces"]) if i not in (0,23)])
G.add_edges_from(connections)
pos = nx.layout.circular_layout(G)
nx.draw_networkx_nodes(G, pos, node_color='#66a3ff', node_size=50) # draw nodes only
nx.draw_networkx_labels(G, pos ) # add labels to the nodes
nx.draw_networkx_edges(G, pos,  edge_color='#e6e6e6') # draw all edges
nx.draw_networkx_edges(G, pos,  edgelist=con_list, edge_color='#ffa31a', width=2)